In [4]:
library(mlbench)
data(BreastCancer)
table(BreastCancer$Class)
library(caret)


   benign malignant 
      458       241 

Warning message:
"package 'caret' was built under R version 3.6.2"Loading required package: lattice
Loading required package: ggplot2
Warning message:
"package 'ggplot2' was built under R version 3.6.2"

In [10]:
x <- upSample(subset(BreastCancer, select = Class), BreastCancer$Class)
y <- downSample(subset(BreastCancer, select = Class), BreastCancer$Class)
table(x)
table(y)

           Class
Class       benign malignant
  benign       458         0
  malignant      0       458

           Class
Class       benign malignant
  benign       241         0
  malignant      0       241

In [13]:
library(party)
str(BreastCancer)
head(BreastCancer)
data <- subset(BreastCancer, select = -Id)

'data.frame':	699 obs. of  11 variables:
 $ Id             : chr  "1000025" "1002945" "1015425" "1016277" ...
 $ Cl.thickness   : Ord.factor w/ 10 levels "1"<"2"<"3"<"4"<..: 5 5 3 6 4 8 1 2 2 4 ...
 $ Cell.size      : Ord.factor w/ 10 levels "1"<"2"<"3"<"4"<..: 1 4 1 8 1 10 1 1 1 2 ...
 $ Cell.shape     : Ord.factor w/ 10 levels "1"<"2"<"3"<"4"<..: 1 4 1 8 1 10 1 2 1 1 ...
 $ Marg.adhesion  : Ord.factor w/ 10 levels "1"<"2"<"3"<"4"<..: 1 5 1 1 3 8 1 1 1 1 ...
 $ Epith.c.size   : Ord.factor w/ 10 levels "1"<"2"<"3"<"4"<..: 2 7 2 3 2 7 2 2 2 2 ...
 $ Bare.nuclei    : Factor w/ 10 levels "1","2","3","4",..: 1 10 2 4 1 10 10 1 1 1 ...
 $ Bl.cromatin    : Factor w/ 10 levels "1","2","3","4",..: 3 3 3 3 3 9 3 3 1 2 ...
 $ Normal.nucleoli: Factor w/ 10 levels "1","2","3","4",..: 1 2 1 7 1 7 1 1 1 1 ...
 $ Mitoses        : Factor w/ 9 levels "1","2","3","4",..: 1 1 1 1 1 1 1 1 5 1 ...
 $ Class          : Factor w/ 2 levels "benign","malignant": 1 1 1 1 1 2 1 1 1 1 ...


Id,Cl.thickness,Cell.size,Cell.shape,Marg.adhesion,Epith.c.size,Bare.nuclei,Bl.cromatin,Normal.nucleoli,Mitoses,Class
1000025,5,1,1,1,2,1,3,1,1,benign
1002945,5,4,4,5,7,10,3,2,1,benign
1015425,3,1,1,1,2,2,3,1,1,benign
1016277,6,8,8,1,3,4,3,7,1,benign
1017023,4,1,1,3,2,1,3,1,1,benign
1017122,8,10,10,8,7,10,9,7,1,malignant


In [23]:
library(rpart)
parts <- createDataPartition(data$Class , p = 0.8)
data.train <- data[parts$Resample1, ]
data.test <- data[-parts$Resample1, ]
(m <- rpart(Class ~ ., data = data.train))

n= 560 

node), split, n, loss, yval, (yprob)
      * denotes terminal node

1) root 560 193 benign (0.65535714 0.34464286)  
  2) Cell.shape=1,2,3 368  20 benign (0.94565217 0.05434783)  
    4) Bare.nuclei=1,2,3,4,5 355   9 benign (0.97464789 0.02535211)  
      8) Normal.nucleoli=1,2,3,6 348   4 benign (0.98850575 0.01149425) *
      9) Normal.nucleoli=4,5,8,10 7   2 malignant (0.28571429 0.71428571) *
    5) Bare.nuclei=6,7,10 13   2 malignant (0.15384615 0.84615385) *
  3) Cell.shape=4,5,6,7,8,9,10 192  19 malignant (0.09895833 0.90104167)  
    6) Cell.size=1,2 10   3 benign (0.70000000 0.30000000) *
    7) Cell.size=3,4,5,6,7,8,9,10 182  12 malignant (0.06593407 0.93406593) *

In [25]:
confusionMatrix(data.test$Class, predict(m, newdata = data.test, type = "class"))

Confusion Matrix and Statistics

           Reference
Prediction  benign malignant
  benign        87         4
  malignant      2        46
                                         
               Accuracy : 0.9568         
                 95% CI : (0.9084, 0.984)
    No Information Rate : 0.6403         
    P-Value [Acc > NIR] : <2e-16         
                                         
                  Kappa : 0.9055         
                                         
 Mcnemar's Test P-Value : 0.6831         
                                         
            Sensitivity : 0.9775         
            Specificity : 0.9200         
         Pos Pred Value : 0.9560         
         Neg Pred Value : 0.9583         
             Prevalence : 0.6403         
         Detection Rate : 0.6259         
   Detection Prevalence : 0.6547         
      Balanced Accuracy : 0.9488         
                                         
       'Positive' Class : benign         
                   

In [26]:
data.up.train <- upSample(subset(data.train, select = -Class), data.train$Class)
(m1 <- rpart(Class ~ ., data = data.up.train))

n= 734 

node), split, n, loss, yval, (yprob)
      * denotes terminal node

 1) root 734 367 benign (0.50000000 0.50000000)  
   2) Cell.size=1,2 347  15 benign (0.95677233 0.04322767)  
     4) Cl.thickness=1,2,3,4,5,6 335   4 benign (0.98805970 0.01194030) *
     5) Cl.thickness=7,8,9,10 12   1 malignant (0.08333333 0.91666667) *
   3) Cell.size=3,4,5,6,7,8,9,10 387  35 malignant (0.09043928 0.90956072)  
     6) Cell.shape=1,2 26  10 benign (0.61538462 0.38461538)  
      12) Bl.cromatin=1,2,3,5 15   0 benign (1.00000000 0.00000000) *
      13) Bl.cromatin=4,7,8 11   1 malignant (0.09090909 0.90909091) *
     7) Cell.shape=3,4,5,6,7,8,9,10 361  19 malignant (0.05263158 0.94736842) *

In [28]:
confusionMatrix(data.test$Class, predict(m1, data.test, type ="class"))

Confusion Matrix and Statistics

           Reference
Prediction  benign malignant
  benign        86         5
  malignant      3        45
                                          
               Accuracy : 0.9424          
                 95% CI : (0.8897, 0.9748)
    No Information Rate : 0.6403          
    P-Value [Acc > NIR] : <2e-16          
                                          
                  Kappa : 0.874           
                                          
 Mcnemar's Test P-Value : 0.7237          
                                          
            Sensitivity : 0.9663          
            Specificity : 0.9000          
         Pos Pred Value : 0.9451          
         Neg Pred Value : 0.9375          
             Prevalence : 0.6403          
         Detection Rate : 0.6187          
   Detection Prevalence : 0.6547          
      Balanced Accuracy : 0.9331          
                                          
       'Positive' Class : benign          

In [31]:
library(caret)
set.seed(137)
train.idx <- createDataPartition(iris$Species, p = .8)[[1]]
data.train <- iris[train.idx, ]
data.test <- iris[-train.idx, ]
(m <- train(Species ~ ., data = data.train, preProcess = c("pca"), method = "rf", 
           ntree = 1000, trControl = trainControl(method = "cv", number = 10, repeats = 3)))

Warning message:
"`repeats` has no meaning for this resampling method."Warning message in randomForest.default(x, y, mtry = param$mtry, ...):
"invalid mtry: reset to within valid range"Warning message in randomForest.default(x, y, mtry = param$mtry, ...):
"invalid mtry: reset to within valid range"Warning message in randomForest.default(x, y, mtry = param$mtry, ...):
"invalid mtry: reset to within valid range"Warning message in randomForest.default(x, y, mtry = param$mtry, ...):
"invalid mtry: reset to within valid range"Warning message in randomForest.default(x, y, mtry = param$mtry, ...):
"invalid mtry: reset to within valid range"Warning message in randomForest.default(x, y, mtry = param$mtry, ...):
"invalid mtry: reset to within valid range"Warning message in randomForest.default(x, y, mtry = param$mtry, ...):
"invalid mtry: reset to within valid range"Warning message in randomForest.default(x, y, mtry = param$mtry, ...):
"invalid mtry: reset to within valid range"Warning message i

Random Forest 

120 samples
  4 predictor
  3 classes: 'setosa', 'versicolor', 'virginica' 

Pre-processing: principal component signal extraction (4), centered (4),
 scaled (4) 
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 108, 108, 108, 108, 108, 108, ... 
Resampling results across tuning parameters:

  mtry  Accuracy   Kappa 
  2     0.9000000  0.8500
  3     0.8916667  0.8375
  4     0.9000000  0.8500

Accuracy was used to select the optimal model using the largest value.
The final value used for the model was mtry = 2.

In [33]:
confusionMatrix(predict(m, newdata = data.test, type = "raw"), data.test$Species)

Confusion Matrix and Statistics

            Reference
Prediction   setosa versicolor virginica
  setosa         10          0         0
  versicolor      0          7         1
  virginica       0          3         9

Overall Statistics
                                          
               Accuracy : 0.8667          
                 95% CI : (0.6928, 0.9624)
    No Information Rate : 0.3333          
    P-Value [Acc > NIR] : 2.296e-09       
                                          
                  Kappa : 0.8             
                                          
 Mcnemar's Test P-Value : NA              

Statistics by Class:

                     Class: setosa Class: versicolor Class: virginica
Sensitivity                 1.0000            0.7000           0.9000
Specificity                 1.0000            0.9500           0.8500
Pos Pred Value              1.0000            0.8750           0.7500
Neg Pred Value              1.0000            0.8636           0.9444
P

In [57]:
data(iris)
data <- iris[, c(1, 2, 5)]
data$Species <- factor(ifelse(data$Species == "setosa", "rare", "common"))
table(data$Species)


common   rare 
   100     50 

In [58]:
library(DMwR)
newData <- SMOTE(Species ~ ., data, perc.over = 600, perc.under=100)
table(newData$Species)


common   rare 
   300    350 